In [118]:
import pandas as pd
import re
import matplotlib.pyplot as plt

from copy import deepcopy




## Количество ожидающих посадки на рейс ⛄ (pass_count)

In [89]:
df_temp = pd.read_excel("05.2022_Пассажиропоток.xlsx")
df_temp.head()

,Дата рейса,Рейс,Вход в чистую зону,Выход на рейс,Терминал,Авиакомпания,Направление куда летит
0,2022-05-01,5N 237,2022-04-30 21:46:20,2022-05-01 00:21:00,B,5N,OVB
1,2022-05-01,5N 237,2022-04-30 21:46:47,2022-05-01 00:21:00,B,5N,OVB
2,2022-05-01,5N 237,2022-04-30 21:47:47,2022-05-01 00:21:00,B,5N,OVB
3,2022-05-01,5N 237,2022-04-30 21:48:20,2022-05-01 00:21:00,B,5N,OVB
4,2022-05-01,5N 237,2022-04-30 21:49:12,2022-05-01 00:21:00,B,5N,OVB


In [90]:
def preprocess_passtream(df: pd.DataFrame) -> pd.DataFrame:
    data = df.copy()

    data["Вход в чистую зону"] = data["Вход в чистую зону"].dt.ceil('30T')
    data["Выход на рейс"] = data["Выход на рейс"].dt.floor('30T')

    data = data[data["Терминал"] == "B"]

    data.drop(columns=["Терминал", "Авиакомпания", "Дата рейса"], inplace=True)

    data.rename(columns={
        "Рейс": "flight", "Вход в чистую зону": "entrance_time",
        "Выход на рейс": "departure_time",
        "Направление куда летит": "destination"
        }, inplace=True)
    
    data.set_index(["entrance_time", "departure_time"], inplace=True)
    data.reset_index(inplace=True)

    return data

In [91]:
def count_passengers(data):
    pass_count = pd.DataFrame({"time": pd.date_range(
        start="2022-05-01 00:00:00", end="2022-05-31 23:30:00", freq="30min"
    )})


    def moment_count(ts: pd.Timestamp):
        return len(data[(data["entrance_time"] <= ts) & (data["departure_time"] >= ts)])
            
    pass_count["num_passengers"] = pass_count.apply(lambda x: moment_count(x["time"]), axis=1)
    
    return pass_count

In [92]:
data = preprocess_passtream(df_temp)
data

,entrance_time,departure_time,flight,destination
0,2022-04-30 22:00:00,2022-05-01 00:00:00,5N 237,OVB
1,2022-04-30 22:00:00,2022-05-01 00:00:00,5N 237,OVB
2,2022-04-30 22:00:00,2022-05-01 00:00:00,5N 237,OVB
3,2022-04-30 22:00:00,2022-05-01 00:00:00,5N 237,OVB
4,2022-04-30 22:00:00,2022-05-01 00:00:00,5N 237,OVB
...,...,...,...,...
42346,2022-05-30 23:30:00,2022-05-31 04:00:00,SU 1592,AER
42347,2022-05-30 23:30:00,2022-05-31 04:00:00,SU 1592,AER
42348,2022-05-30 23:30:00,2022-05-31 04:00:00,SU 1592,AER
42349,2022-05-31 00:00:00,2022-05-31 04:00:00,SU 1592,AER


In [93]:
pass_count = count_passengers(data)
pass_count.head()

,time,num_passengers
0,2022-05-01 00:00:00,1707
1,2022-05-01 00:30:00,1124
2,2022-05-01 01:00:00,701
3,2022-05-01 01:30:00,297
4,2022-05-01 02:00:00,148


## Температура в этот день/вечер ⛅ (heat_data)

In [94]:
df_temp = pd.read_excel("heat_05_2022.xlsx")
df_temp

,first,first_in_group,first_in_group 2
0,1,13,10
1,2,14,9
2,3,18,14
3,4,8,5
4,5,9,7
5,6,16,12
6,7,18,15
7,8,19,11
8,9,7,7
9,10,6,11


In [95]:
heat_data = df_temp.rename(columns={
        "first": "day", "first_in_group": "heat_day",
        "first_in_group 2": "heat_night"
        })
heat_data.head()

,day,heat_day,heat_night
0,1,13,10
1,2,14,9
2,3,18,14
3,4,8,5
4,5,9,7


# Преобразование начального датасета 🚧

In [119]:
df_temp = pd.read_excel("05.2022_Выручка.xlsx")
df_temp.head()

,TimeThirty,DAT,Торговая точка,Касса,orgtype,terminal,tzone,TotalSumm
0,2022-05-01 00:00:00.000,2022-05-01,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,755 448
1,2022-05-04 13:30:00.000,2022-05-04,Торговая точка 2373,3114,Общепит,B,Общедоступная зона-вылет,46 800
2,2022-05-05 18:00:00.000,2022-05-05,Торговая точка 8998,9887,Торговля,B,Чистая зона ВВЛ-вылет,160 160
3,2022-05-25 22:00:00.000,2022-05-25,Торговая точка 8313,2251,Общепит,B,Чистая зона ВВЛ-вылет,248 960
4,2022-05-10 00:30:00.000,2022-05-10,Торговая точка 8998,9887,Торговля,B,Чистая зона ВВЛ-вылет,117 600


In [120]:
df_temp.dtypes

TimeThirty        object
DAT               object
Торговая точка    object
Касса              int64
orgtype           object
terminal          object
tzone             object
TotalSumm         object
dtype: object

In [124]:
df_0601 = deepcopy(df_temp[df_temp['Торговая точка'] == "Торговая точка 0601"])
df_0601

,TimeThirty,DAT,Торговая точка,Касса,orgtype,terminal,tzone,TotalSumm
0,2022-05-01 00:00:00.000,2022-05-01,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,755 448
7,2022-05-04 17:30:00.000,2022-05-04,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,"735 549,6"
12,2022-05-04 04:30:00.000,2022-05-04,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,"90 546,4"
40,2022-05-31 01:00:00.000,2022-05-31,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,411 600
43,2022-05-24 05:30:00.000,2022-05-24,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,1 012 096
...,...,...,...,...,...,...,...,...
11814,2022-05-25 17:30:00.000,2022-05-25,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,"885 608,8"
11820,2022-05-20 23:30:00.000,2022-05-20,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,523 096
11821,2022-05-07 06:30:00.000,2022-05-07,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,1 818 016
11827,2022-05-12 14:30:00.000,2022-05-12,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,"948 813,6"


In [125]:
df_0601['TimeThirty'] = df_0601['TimeThirty'].apply(pd.to_datetime)
df_0601.dtypes

TimeThirty        datetime64[ns]
DAT                       object
Торговая точка            object
Касса                      int64
orgtype                   object
terminal                  object
tzone                     object
TotalSumm                 object
dtype: object

In [128]:
df_0601 = df_0601.sort_values(by=['TimeThirty'])
df_0601

,TimeThirty,DAT,Торговая точка,Касса,orgtype,terminal,tzone,TotalSumm
0,2022-05-01 00:00:00,2022-05-01,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,755 448
2696,2022-05-01 00:30:00,2022-05-01,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,1 298 616
2573,2022-05-01 01:00:00,2022-05-01,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,451 200
11515,2022-05-01 01:30:00,2022-05-01,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,276 000
3370,2022-05-01 02:30:00,2022-05-01,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,68 000
...,...,...,...,...,...,...,...,...
5191,2022-05-31 21:30:00,2022-05-31,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,881 320
696,2022-05-31 22:00:00,2022-05-31,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,"1 568 435,2"
10031,2022-05-31 22:30:00,2022-05-31,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,1 067 360
7747,2022-05-31 23:00:00,2022-05-31,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,"571 071,2"


In [137]:
df_0601['TotalSumm'].apply(pd.to_numeric)

ValueError: Unable to parse string "755 448" at position 0

In [143]:
for i in df_0601['TotalSumm']:
    print(i)
    print(int(i.split(',')[0].replace(' ','').replace('\xa0','')))

755 448
755448
1 298 616
1298616
451 200
451200
276 000
276000
68 000
68000
182 400
182400
675 120
675120
349 816
349816
1 093 464
1093464
2 308 792,8
2308792
740 825,6
740825
1 538 000
1538000
507 625,6
507625
1 942 877,6
1942877
2 310 200
2310200
1 182 785,6
1182785
2 112 260,8
2112260
1 077 683,2
1077683
93 386,4
93386
998 225,6
998225
1 215 360
1215360
1 355 424
1355424
722 461,6
722461
924 240
924240
1 316 000
1316000
2 860 960
2860960
635 200
635200
135 562,4
135562
144 840
144840
187 368
187368
1 450 199,2
1450199
1 711 484,8
1711484
1 487 743,2
1487743
808 207,2
808207
5 740 761,6
5740761
1 398 616,8
1398616
966 720
966720
396 356
396356
474 400
474400
308 693,6
308693
201 306,4
201306
41 600
41600
195 200
195200
65 600
65600
184 320
184320
519 382,4
519382
806 400
806400
969 992
969992
1 378 153,6
1378153
1 281 487,2
1281487
459 692,8
459692
959 492,8
959492
1 570 696,8
1570696
1 722 984,8
1722984
2 029 200
2029200
1 256 808
1256808
1 735 605,6
1735605
1 087 348,8
1087348
56 4

748 000
748000
790 008
790008
444 164,8
444164
1 655 799,2
1655799
454 400
454400
1 288 922,4
1288922
3 199 721,6
3199721
1 595 047,2
1595047
1 147 108,8
1147108
1 288 510,4
1288510
1 556 728
1556728
1 464 328
1464328
891 588,8
891588
168 869,6
168869
1 670 160
1670160
284 561,6
284561
1 843 324
1843324
1 304 176,8
1304176
2 136 043,2
2136043
1 033 600
1033600
1 527 135,2
1527135
1 129 280
1129280
641 200
641200
1 382 241,6
1382241
731 360
731360
843 200
843200
727 496
727496
365 880
365880
171 200
171200
196 556,8
196556
42 000
42000
370 208
370208
90 400
90400
183 200
183200
666 379,2
666379
630 800
630800
1 320 991,2
1320991
688 065,6
688065
1 063 230,4
1063230
815 792
815792
1 803 062,4
1803062
1 786 265,6
1786265
1 463 820
1463820
1 316 872
1316872
836 427,2
836427
1 385 600
1385600
1 112 440
1112440
772 334,4
772334
1 198 400
1198400
2 158 084,8
2158084
900 219,2
900219
2 453 200
2453200
814 152
814152
1 426 552
1426552
1 203 200
1203200
2 601 730,4
2601730
2 674 964
2674964
1 53

In [140]:
print([-1] * 10 + list(df_0601['TotalSumm'].apply(lambda x: int(x.replace(' ','').replace('\xa0',''))))) #убрать преобразование

ValueError: invalid literal for int() with base 10: '2308792,8'

In [131]:
for lag in range(48):
    df_0601[f'lag_{lag}'] = [-1] * lag + [df_0601['TotalSumm'][:-lag]]
    print([-1] * lag + [df_0601['TotalSumm'][:-lag]])
df_0601

ValueError: Length of values (1) does not match length of index (1378)

# Проверка

In [97]:
data["flight"].unique()[0].replace(' ','')

'5N237'

In [98]:
#внутренние направления
internal_dest = {
    "ABA", "ADH", "AMV", "DYR", "AAQ", "KVK", "ULAH", "ARH", "ASF", "ACS",
    "UIUB", "BAX", "EGO", "BCX", "UNBI", "UHMK", "BQS", "TGP", "BTK", "BZK",
    "UUA", "UWWB", "ULDW", "VLU", "VUS", "VVO", "OGZ", "UUBL", "VOG", "VGD",
    "VKT", "UUOD", "VOZ", "UUYK", "GDZ", "UNBG", "GRV", "DKS", "SVX", "USSK",
    "EIE", "UEVV", "IWA", "IAA", "IJK", "INA", "IKT", "JOK", "KZN", "KGD",
    "ULDT", "KEJ", "KVX", "KGP", "UHKD", "KXK", "ULPM", "KMW", "KSZ", "KRR",
    "URKN", U"IAE", "KJA", "UNKM", "UWSK", "KRO", "BVV", "URS", "KYZ", "LDG",
    "LPK", "UHMT", "GDX", "GDG", "MQF", "MCX", "ULAE", "MRV", "MJZ", "BKA",
    "VKO", "DME", "UUMO", "UUBW", "СОЦ", "SVO", "MMK", "UHMI", "NYM", "NAL",
    "NNM", "CNN", "UIUN", "NJC", "NBC", "GOJ", "НИК", "NOZ", "UNNE", "OVB",
    "NUX", "NGL", "NSK", "NOJ", "USHN", "OKT", "UNOS", "OMS", "ULAO", "OEL",
    "REN", "OSW", "OHH", "OHO", "PWE", "PEZ", "PEE", "PES", "ULPP", "PKC",
    "PEX", "PVS", "PKV", "ULLP", "RAT", "ROV", "RYB", "RZN", "RZN", "SLY",
    "KUF", "UWWS", "LED", "SKX", "RTW", "UWUA", "LNX", "GVN", "CSH", "AER",
    "STW", "UUOS", "UWUS", "SGC", "SCW", "URRT", "TBW", "UUBN", "UUEM", "IKS",
    "UHPT", "TOX", U"WWT", "TOF", "UNIT", "TYD", U"STL", "TJM", "PYJ", "UUD",
    "UIUW", "ULY", "UWLW", "UKX", "UEMT", "USK", "UFA", "UCT", "UHHT", "KHV",
    "UEMH", "HMA", "ХАУ", "HTG", "VLK", "UIAR", "CSY", "CEK", "CEE", "CYX",
    "HTA", "CKH", "EKS", "CKL", "ESL", "DEE", "UUS", "YKS", "USMY", "UUBX", "IAR"
}

In [99]:
for i in data["destination"].unique():
    print(i in internal_dest)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True


In [115]:
(pass_count['time'].apply(lambda x: x.hour) > 1) & (pass_count['time'].apply(lambda x: x.hour) < 23)

0       False
1       False
2       False
3       False
4        True
        ...  
1483     True
1484     True
1485     True
1486    False
1487    False
Name: time, Length: 1488, dtype: bool